# Rule Splitting & ElasticSearch Uploads
## LMI Capstone Team
## Summer Chambers | Steve Morris | Kaleb Shikur

### Setup ES

In [1]:
'''
Rule Splitting, Uploading to ES
'''
import pandas as pd
from elasticsearch import Elasticsearch, RequestsHttpConnection, ElasticsearchException
# from requests_aws4auth import AWS4Auth
import regex as re
from bs4 import BeautifulSoup
import numpy as np
import requests #gets urls
import time
# from aws_requests_auth.boto_utils import BotoAWSRequestsAuth

host = 'https://search-lmi-capstone-test-bbewbwsli4uchb7r24zujwrqne.us-east-1.es.amazonaws.com'
# auth = BotoAWSRequestsAuth(aws_host=host,
#                            aws_region='us-east-1',
#                            aws_service='es')
# awsauth = AWS4Auth(YOUR_ACCESS_KEY, YOUR_SECRET_KEY, REGION, 'es')

In [2]:
es = Elasticsearch(host, timeout = 45)

# es = Elasticsearch(
#     hosts=[{'host': host, 'port': 443}],
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
print(es.info())

# es = Elasticsearch(
#     hosts=[{'host': host, 'port': 443}],
#     http_auth=auth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
# print(es.info())

{'name': 'dedbbb462bf60acdb6c63621c8bf117c', 'cluster_name': '846033058400:lmi-capstone-test', 'cluster_uuid': 'vhqZptiWQEq0BVgaN6qi8Q', 'version': {'number': '7.8.0', 'build_flavor': 'oss', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2020-10-15T22:09:28.519938Z', 'build_snapshot': False, 'lucene_version': '8.5.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


### Define Rule Splits

In [3]:
rule_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Rule/CMS-2018-0101-0001.txt"

In [4]:
def splitRule_headers(rule_url):

    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    
    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    rulechunks = {}
    
    startdict = {'a2':['2. proposals for modified participation options under 5-year agreement periods', '3. creating a basic track with glide path to performance-based risk'], \
     'a3':['3. creating a basic track with glide path to performance-based risk', '4. permitting annual participation elections'], \
    'a4b':['b. proposals for permitting election of differing levels of risk within the basic track\'s glide path', 'c. proposals for permitting annual election of beneficiary assignment methodology'], \
    'a4c':['c. proposals for permitting annual election of beneficiary assignment methodology', '5. determining participation options based on medicare ffs revenue and prior participation '], \
    'a5b':['b. differentiating between low revenue acos and high revenue acos', 'c. determining participation options based on prior participation of aco legal entity and aco participants'], \
    'a5c':['c. determining participation options based on prior participation of aco legal entity and aco participants', 'd. monitoring for financial performance'], \
    'a5d': ['d. monitoring for financial performance', '6. requirements for aco participation in two-sided models'], \
    'a6b': ['b. election of msr/mlr by acos', 'c. aco repayment mechanisms'], \
    'a6c': ['c. aco repayment mechanisms', 'd. advance notice for and payment consequences of termination '], \
    'a6d2': ['(2) proposals for advance notice of voluntary termination', '(3) proposals for payment consequences of termination'], \
    'a6d3': ['(3) proposals for payment consequences of termination', '7. participation options for agreement periods beginning in 2019'], \
    'a7b': ['b. methodology for determining financial and quality performance for the 6-month performance years during 2019', 'c. applicability of program policies to acos participating in a 6-month performance year'], \
    'a7c': ['c. applicability of program policies to acos participating in a 6-month performance year', 'b. fee-for-service benefit enhancements'], \
    'b2a': ['a. shared savings program snf 3-day rule waiver', 'b. billing and payment for telehealth services'], \
    'b2b': ['b. billing and payment for telehealth services', 'c. providing tools to strengthen beneficiary engagement'], \
    'c2': ['2. beneficiary incentives', '3. empowering beneficiary choice'], \
    'c3a': ['3. empowering beneficiary choice', 'b. beneficiary opt-in based assignment methodology'], \
    'c3b': ['b. beneficiary opt-in based assignment methodology', 'd. benchmarking methodology refinements'],  \
    'd2': ['2. risk adjustment methodology for adjusting historical benchmark each performance year', '3. use of regional factors when establishing and resetting acos\' benchmarks'], \
    'd3b': ['b. proposals to apply regional expenditures in determining the benchmark for an aco\'s first agreement period', 'c. proposals for modifying the regional adjustment'], \
    'd3c': ['c. proposals for modifying the regional adjustment', 'd. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark'], \
    'd3d': ['d. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark', '4. technical changes to incorporate references to benchmark rebasing policies'], \
    'd4': ['4. technical changes to incorporate references to benchmark rebasing policies', 'e. updating program policies'], \
    'e2': ['2. revisions to policies on voluntary alignment', '3. revisions to the definition of primary care services used in beneficiary assignment'], \
    'e3': ['3. revisions to the definition of primary care services used in beneficiary assignment', '4. extreme and uncontrollable circumstances policies for the shared savings program'], \
    'e4': ['4. extreme and uncontrollable circumstances policies for the shared savings program', '5. program data and quality measures'], \
    'e5':['5. program data and quality measures', '6. promoting interoperability'], \
    'e6':['6. promoting interoperability', '7. coordination of pharmacy care for aco beneficiaries'], \
    'e7':['7. coordination of pharmacy care for aco beneficiaries', 'f. applicability of proposed policies to track 1+ model acos'], \
    'f2':['2. unavailability of application cycles for entry into the track 1+ model in 2019 and 2020', '3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements'], \
    'f3':['3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements', 'g. summary of proposed timing of applicability']}
    
    for key, value in startdict.items():    
       splitlist = splitlist.split(value[0]) #split on start of desired section
       split_further = splitlist[1].split(value[1]) #split again on start of undesired section
       rulechunks[key] = {"text": value[0]+split_further[0]} #choose only first half to upload to dict
       splitlist = splitlist[1] #choose second half to prepare for next split
    
    #print(rulechunks.keys())   
    #print(rulechunks["f3"])
    
    #lengths = [len(chunk) for chunk in rulechunks.values()]
    #print(lengths) #characters
    
    return rulechunks

In [5]:
#Updated Character-Count Split

def splitRule_line_chars(rule_url):
    #get rule
    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings

    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    #Split on new paragraphs
    paragraphs = splitlist.split('\r\n')
    
    #add new lines while under 2000 characters
    for i in range(len(paragraphs) - 1):
        while i < (len(paragraphs)-1) and len(paragraphs[i]) < 2000:
            paragraphs[i] += paragraphs[i+1]
            del(paragraphs[i+1])

    #add to dictionary
    rulechunks = {}
    keys = range(len(paragraphs))
    for i in keys:
        rulechunks[i] = {"text": paragraphs[i]}

    return rulechunks

In [6]:
len(splitRule_line_chars(rule_url))

329

In [7]:
def splitRule_chars(rule_url):
    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    
    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    #chunk by num chars
    #chunklist = []
    
    chunklist = [(splitlist[i:i+8000]) for i in range(0, len(splitlist), 8000)]
    
    #make dict
    rulechunks = {}
    keys = range(len(chunklist))
    for i in keys:
        rulechunks[i] = {"text": chunklist[i]}

    return rulechunks

In [8]:
def splitRule_headchars(rule_url):
    chunks = splitRule_headers(rule_url) #call first method
    for key, value in chunks.items():
        if len(value["text"]) > 50000: #check to see if chunk is really long
            smalldict = {}
            chunklist = [value["text"][i:i+25000] for i in range(0, len(value["text"]), 25000)] #split chunk
            keys = range(len(chunklist))
            for i in keys:
                smalldict[key+str(i)] = {"text": chunklist[i]} #add to sub-dictionary
            chunks[key] = smalldict #add to big dictionary
        else:
            chunks[key] = {key+str(0): value} #add to big dictionary
    return chunks

### Uploading Rule Splits to ES

In [ ]:
def rulesplit_toES(rulechunks, es_index, idnum):
    for key, value in rulechunks.items():
            res = es.index(index=es_index, id=idnum, body={key:value}, doc_type='_doc')
            idnum += 1
            es.indices.refresh(index=es_index)
    print(f"Last id uploaded: {idnum-1}")

In [ ]:
def ruletoES_hybrid(rulechunks, es_index, idnum):
    for key, value in rulechunks.items():
        for small_key, small_value in value.items():
            res = es.index(index=es_index, id=idnum, body={small_key:small_value})
            idnum += 1
            es.indices.refresh(index=es_index)
    print(f"Last id uploaded: {idnum-1}")

In [ ]:
rulesplit_test = splitRule_headers(rule_url)
test_upload = rulesplit_toES(rulesplit_test, "headers_new", 1)

In [ ]:
rulesplit_test2 = splitRule_chars(rule_url)
test_upload2 = rulesplit_toES(rulesplit_test2, "char_8000", 1)

In [ ]:
rulesplit_test3 = splitRule_headchars(rule_url)
test_upload3 = ruletoES_hybrid(rulesplit_test3, "hybrid_new", 1)

### Querying ES

In [9]:
#ES Search
def ES_search(index, querydict):
    search = es.search(index=index, doc_type="_doc", body={"query": querydict})
    test_list = {}
    if search['hits']['hits'] != []:
        for h in search['hits']['hits']:
            key = list((h['_source'].keys()))
            test_list[key[0]]=h['_score']
    return test_list

In [10]:
test_query = {"simple_query_string": {"query": "urban"}}

In [11]:
headers_test_search = ES_search("headers_new", test_query)
headers_test_search

{'a6c': 0.45770907, 'a4c': 0.45549875, 'b2b': 0.29446962, 'c2': 0.2554381}

In [10]:
char_test_search = ES_search("char_8000", test_query)
char_test_search

{'64': 0.29684716, '70': 0.29528618, '68': 0.29148874}

In [12]:
hybrid_test_search = ES_search("hybrid_new", test_query)
hybrid_test_search

{'c20': 0.40307403,
 'a4c15': 0.29141787,
 'a6c7': 0.29115358,
 'a6c8': 0.29011896,
 'a4c16': 0.28976172,
 'a4c17': 0.289463,
 'a6c9': 0.28831965,
 'b2b0': 0.1865641}

### More Utilities:

In [12]:
# Delete an index:
# es.indices.delete(index='index_name', ignore=[400, 404])

### Retrieve Comment Documents

In [13]:
comment_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Comments/"

In [14]:
def retrieve_comments(comment_url):
    comment_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Comments/"
    response = requests.get(comment_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    a_tags = soup.findAll("a")
    links = [tag["href"] for tag in a_tags]
    txt_links = [link for link in links if '.txt' in link]
    comments = {}
    for suffix in txt_links:
        comments[suffix] = requests.get(comment_url+suffix).text.lower()
        #print(f"scraping comment {suffix}")
    print(f"scraped {len(txt_links)} comments")
    return comments

In [16]:
comments2018 = retrieve_comments(comment_url)

scraped 674 comments


In [17]:
comment_lengths = {key: len(value) for key, value in comments2018.items()}

In [18]:
sorted_comment_lengths = dict(sorted(comment_lengths.items(), key=lambda x: x[1]))

In [19]:
comments_short = {key: len(value) for key, value in comments2018.items() if "attach" in value and len(value) < 200}

In [20]:
comments2018 = {(key[14:20]): value for key, value in comments2018.items()}
sorted_keys = sorted(list(comments2018.keys()))

# now for each key in the list
for i in range(len(comments2018)-1):
    # get key at index i and key at index i+1 and compare them
    if sorted_keys[i+1][0:4] == sorted_keys[i][0:4]:
        comments2018[sorted_keys[i+1]] = comments2018[sorted_keys[i]] + comments2018[sorted_keys[i+1]]
        del(comments2018[sorted_keys[i]])

#{key: len(value) for key, value in comments2018.items()}

In [21]:
comments2018 = {key[0:4]:value for key, value in comments2018.items()}

In [22]:
len(comments2018)

468

In [23]:
import json
json = json.dumps(comments2018)
f = open('comments2018.json','w')
f.write(json)
f.close()

### Querying Comments

In [27]:
es.indices.refresh(index='headers_new')

{'_shards': {'total': 10, 'successful': 5, 'failed': 0}}

In [25]:
es.indices.refresh(index='hybrid_new')

{'_shards': {'total': 10, 'successful': 5, 'failed': 0}}

In [26]:
test_comment001_query = {"simple_query_string": {"query": comments2018["0002"]}}

In [29]:
ES_search("headers_new", test_comment001_query)

{'a4c': 144.72498,
 'a6c': 142.13535,
 'a5c': 110.61184,
 'c3b': 106.4453,
 'a3': 105.05455,
 'a5b': 104.90532,
 'a2': 103.80479,
 'e4': 100.2969,
 'a7c': 97.574455,
 'a7b': 93.47521}

In [27]:
ES_search("hybrid_new", test_comment001_query)

{'c20': 105.905975,
 'a5b0': 104.90532,
 'a20': 103.80479,
 'e40': 100.2969,
 'a6c19': 99.05433,
 'a4c27': 98.69642,
 'a5c1': 98.16931,
 'a4c7': 97.86486,
 'a4c9': 97.61569,
 'a7c0': 97.574455}

In [28]:
# dict_first3 = {key:value for key,value in comments2018}
dict_first3 = {}
for key in list(comments2018.keys())[0:3]:
    dict_first3[key] = comments2018[key]

In [30]:
# Test HYBRID on dictionary of 3 key/value

results = {}
for key, value in dict_first3.items():
    query =  {"simple_query_string": {"query": value}}
    try:
        search1 = ES_search("hybrid_new", query)
    except ElasticsearchException as es1:
            print(f'maximum clause error at index {key}')
    results[key] = list(search1.keys())
    time.sleep(1)
    es.indices.refresh(index='hybrid_new')
    print(i, end=' ')
    i+=1

675 maximum clause error at index 0003
676 677 

In [66]:
# Test HEADERS on dictionary of 3 key/value

results = {}
for key, value in dict_first3.items():
    query =  {"simple_query_string": {"query": value}}
    try:
        search1 = ES_search("headers_new", query)
    except ElasticsearchException as es1:
            print(f'maximum clause error at index {key}')
    results[key] = list(search1.keys())
    time.sleep(1)
    es.indices.refresh(index='headers_new')
    print(i, end=' ')
    i+=1

In [32]:
# HYBRID Full dictionary Query

results = {}
i=0
for key, value in comments2018.items():
    query =  {"simple_query_string": {"query": value}}
    try:
        search1 = ES_search("hybrid_new", query)
    except ElasticsearchException as es1:
            print(f'maximum clause error at index {key}')
    results[key] = list(search1.keys())
    time.sleep(1)
    es.indices.refresh(index='hybrid_new')
    print(i, end=' ')
    i+=1

0 1 2 3 4 5 6 7 8 9 maximum clause error at index 0012
10 maximum clause error at index 0013
11 maximum clause error at index 0014
12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 maximum clause error at index 0055
53 maximum clause error at index 0056
54 maximum clause error at index 0057
55 maximum clause error at index 0058
56 maximum clause error at index 0059
57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 maximum clause error at index 0081
79 maximum clause error at index 0082
80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 maximum clause error at index 0138
136 137 maximum clause error at index 0140
138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 maximum clause error at index 0158
156 maximum clause error at ind

TransportError: TransportError(503, '{\n  "message": "No server available to handle the request",\n}\n')

In [35]:
# Capture HYBRID results in CSV
import csv
w = csv.writer(open('hybrid_results.csv', 'w'))
for key, val in results.items():
    w.writerow([key,val])

In [ ]:
# Capture HYBRID results in JSON
import json
json = json.dumps(results)
f = open('hybrid_results.json','w')
f.write(json)
f.close()

In [ ]:
# HEADERS Full dictionary Query

results = {}
i=0
for key, value in comments2018.items():
    query =  {"simple_query_string": {"query": value}}
    try:
        search1 = ES_search("headers_new", query)
    except ElasticsearchException as es1:
            print(f'maximum clause error at index {key}')
    results[key] = list(search1.keys())
    time.sleep(1)
    es.indices.refresh(index='headers_new')
    print(i, end=' ')
    i+=1

In [35]:
import csv
w = csv.writer(open('header_results.csv', 'w'))
for key, val in results.items():
    w.writerow([key,val])

In [ ]:
import json
json = json.dumps(results)
f = open('header_results.json','w')
f.write(json)
f.close()

In [67]:
results

{'0002': ['a4c', 'a6c', 'a5c', 'c3b', 'a3', 'a5b', 'a2', 'e4', 'a7c', 'a7b'],
 '0003': ['a4c', 'a6c', 'a3', 'a5c', 'e4', 'c3b', 'a5b', 'a2', 'a7c', 'a7b'],
 '0004': ['a4c', 'a6c', 'a5c', 'a3', 'e4', 'a7c', 'a2', 'a5b', 'c3b', 'e6']}